# Importando librerías para el análisis exploratorio de los datos.

### pandas y seaborn, librerías que ayudarán al análisis exploratorio gráfico y manipulación de datos

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargando los datos descargados en formato Excel

In [85]:
gtc_covid = pd.read_excel("/content/concursos-terminados-adjudicados-al-amparo-de-la-emergecia-covid19.xlsx", header="infer", index_col=0, parse_dates=True)
gtc_todos = pd.read_excel("/content/concursos-publicados-2020-adjudicado.xlsx", header="infer",index_col=0,parse_dates=True)

Analizando los tamaños, contenidos y tipos de datos de cada archivo.  


In [86]:
gtc_covid.shape

(4237, 13)

In [87]:
gtc_todos.shape

(60977, 23)

In [83]:
gtc_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4237 entries, 0 to 4236
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FILA               4237 non-null   int64         
 1   NOG_CONCURSO       4237 non-null   int64         
 2   FECHA_PUBLICACION  4237 non-null   datetime64[ns]
 3   FECHA_LIMITE       4237 non-null   datetime64[ns]
 4   TIPO_CONCURSO      4237 non-null   object        
 5   ESTATUS_CONCURSO   4237 non-null   object        
 6   NOMBRE_ENTIDAD     4237 non-null   object        
 7   NOMBRE_UNIDAD      4237 non-null   object        
 8   TITULO_NOG         4237 non-null   object        
 9   NOMBRE_MODALIDAD   4237 non-null   object        
 10  MONTO              4237 non-null   float64       
 11  CANTIDAD           4237 non-null   int64         
 12  CATEGORIA          4237 non-null   object        
 13  NUMERO_DOCUMENTO   4237 non-null   object        
dtypes: datet

In [84]:
gtc_todos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60977 entries, 0 to 60976
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   NO                              60977 non-null  int64         
 1   TIPO DE ENTIDAD PADRE           60977 non-null  object        
 2   TIPO DE ENTIDAD                 60977 non-null  object        
 3   ENTIDAD COMPRADORA              60977 non-null  object        
 4   UNIDAD COMPRADORA               60977 non-null  object        
 5   NOG CONCURSO                    60977 non-null  int64         
 6   DESCRIPCIÓN                     60977 non-null  object        
 7   MODALIDAD                       60977 non-null  object        
 8   SUB MODALIDAD                   5837 non-null   object        
 9   NIT                             60262 non-null  object        
 10  NOMBRE                          60262 non-null  object        
 11  MO

El archivo **gtc_todos** contiene los datos de adjudicación de concursos, mientras que el **gtc_covid**, tiene información básica sin información de la adjudicación. Suponiendo que **gtc_todos** contiene a **gtc_covid**, hacemos un subset de todos con los datos de covid, haciendo un match por NOG (Número de operación de Guatecompras) 

In [ ]:
resultado = gtc_todos[gtc_todos["NOG CONCURSO"].isin(gtc_covid["NOG_CONCURSO"])]

In [ ]:
resultado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4819 entries, 27769 to 60931
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   TIPO DE ENTIDAD PADRE           4819 non-null   object        
 1   TIPO DE ENTIDAD                 4819 non-null   object        
 2   ENTIDAD COMPRADORA              4819 non-null   object        
 3   UNIDAD COMPRADORA               4819 non-null   object        
 4   NOG CONCURSO                    4819 non-null   int64         
 5   DESCRIPCIÓN                     4819 non-null   object        
 6   MODALIDAD                       4819 non-null   object        
 7   SUB MODALIDAD                   4486 non-null   object        
 8   NIT                             4819 non-null   object        
 9   NOMBRE                          4819 non-null   object        
 10  MONTO                           4819 non-null   float64       
 11 

In [ ]:
#resultado.to_excel("salidaFinal.xlsx")
resultado.nunique()

TIPO DE ENTIDAD PADRE                2
TIPO DE ENTIDAD                      5
ENTIDAD COMPRADORA                 164
UNIDAD COMPRADORA                  274
NOG CONCURSO                      4224
DESCRIPCIÓN                       4138
MODALIDAD                            4
SUB MODALIDAD                        1
NIT                                736
NOMBRE                             736
MONTO                             2664
FECHA DE PUBLICACIÓN                90
MES DE PUBLICACIÓN                   4
AÑO DE PUBLICACIÓN                   1
FECHA DE ULTIMA ADJUDICACIÓN        86
FECHA DE ADJUDICACIÓN               86
MES DE ADJUDICACIÓN                  4
AÑO DE ADJUDICACIÓN                  1
CATEGORÍAS                          90
ESTATUS DEL CONCURSO                 1
FECHA CIERRE RECEPCIÓN OFERTAS    1373
MES DE CIERRE RECEPCIÓN              4
AÑO DE CIERRE RECEPCIÓN              1
dtype: int64

### Del listado anterior, donde totalizamos los valores únicos por cada columna, vemos varias cosas:

1. Hay varios campos con valores únicos, candidatos eliminarse para el análisis futuro
2. Hay algunas tipificaciones por Tipo de Entidad por analizar
3. Hay 164 entidades compradoras y 274 unidades compradoras, ambas a niveles distintos.
4. Hay más filas que concursos, ya que un concurso puede adjudicarse a más de un proveedor
5. Hay 4 modalidades de compra, todas relacionadas con la emergencia del COVID19
6. Hay 90 distintas categorías, que luego analizaremos
7. Al haber fecha de recepción de ofertas, puede analizarse el tiempo en el que cada concurso estuvo abierto, y poder detectar anomalías.

In [ ]:
resultado["MODALIDAD"].value_counts()

Procedimientos Regulados por el artículo 44 LCE (Casos de Excepción)    4486
Compra Directa con Oferta Electrónica (Art. 43 LCE Inciso b)             258
Compra Directa con Oferta Electrónica Ley de Emergencia COVID19           72
Cotización (Art. 38 LCE)                                                   3
Name: MODALIDAD, dtype: int64

In [ ]:
resultado[["MODALIDAD", "SUB MODALIDAD"]].nunique()

MODALIDAD        4
SUB MODALIDAD    1
dtype: int64

In [ ]:
resultado["TIPO DE ENTIDAD PADRE"].value_counts()

Sector Público                    4813
Fideicomisos y otras entidades       6
Name: TIPO DE ENTIDAD PADRE, dtype: int64

In [ ]:
resultado["TIPO DE ENTIDAD"].value_counts()

Administración Central                                         3042
Entidades Descentralizadas, Autónomas y de Seguridad Social    1490
Gobiernos Locales (Municipalidades, Mancomunidades, etc.)       270
Empresas Públicas (Nacionales y Municipales)                     11
ONG's, patronatos, comités, asociaciones y fundaciones            6
Name: TIPO DE ENTIDAD, dtype: int64

In [65]:
columnas_eliminar=["AÑO DE PUBLICACIÓN", "AÑO DE ADJUDICACIÓN", "AÑO DE CIERRE RECEPCIÓN", "MES DE PUBLICACIÓN", "MES DE ADJUDICACIÓN", "MES DE CIERRE RECEPCIÓN", "ESTATUS DEL CONCURSO"]
result_red = resultado.drop(axis=1,labels=columnas_eliminar)

In [ ]:
print("Promedio del monto: Q. % 8.2f" % result_red.MONTO.mean())
print("Máximo del monto: Q. % 8.2f" % result_red.MONTO.max())
print("Mínimo del monto: Q. % 8.2f" % result_red.MONTO.min())

Promedio del monto: Q.  115304.34
Máximo del monto: Q.  20224512.00
Mínimo del monto: Q.    40.00


In [ ]:
print("Mínimo de la fecha:" , result_red["FECHA DE ADJUDICACIÓN"].min())
print("Máximo de la fecha:" , result_red["FECHA DE ADJUDICACIÓN"].max())

Mínimo de la fecha: 2020-03-13 00:00:00
Máximo de la fecha: 2020-06-19 00:00:00


## 20 proveedores con más adjudicaciones

In [109]:
top_20_NIT_cant = result_red.NIT.value_counts().head(20)
lista_top_20 = result_red.loc[result_red["NIT"].isin(top_20_NIT_cant.index)][["NOMBRE","NIT"]].drop_duplicates()
print(lista_top_20)

                                                  NOMBRE        NIT
NO                                                                 
27769  COMPAÑIA DE EQUIPO MEDICO-HOSPITALARIO, SOCIED...    4556984
29374             PRODUCTOS SANITARIOS, SOCIEDAD ANONIMA   41274989
29375                      SOLIS,GONZALEZ,,JORGE,ALFREDO    2937913
33528             LABORATORIO RUIPHARMA SOCIEDAD ANONIMA    5490421
33530  SERVICIO DE COMERCIO INTERNACIONAL SOCIEDAD AN...   22339655
33532  DROGUERIA Y DISTRIBUIDORA PHARMAR, SOCIEDAD AN...   93752180
33631                          LABYMED, SOCIEDAD ANONIMA   30428378
34045  ASESORIA EN RECURSOS OCUPACIONALES, SOCIEDAD A...  100737080
34320  EQUIPOS PARA HOSPITALES Y MEDICINAS, SOCIEDAD ...    656058K
34321                DROGUERIA REFASA , SOCIEDAD ANONIMA    1204076
34416        NUEVOS ETICOS NEOETHICALS, SOCIEDAD ANONIMA   39796558
34592                    RODRÍGUEZ,LÓPEZ,,VAYRON,RONALDO   26691035
34628              ACEITUNO,RUBIO,JIMENEZ,JENY,B

## Top 20 de los proveedores en base al Monto



In [126]:
nit_monto = result_red.groupby(["NIT", "NOMBRE"])
print(nit_monto.MONTO.sum().sort_values(ascending=False).head(20))

NIT       NOMBRE                                                     
30428378  LABYMED, SOCIEDAD ANONIMA                                      42139448.65
1700510   PRODUCTOS ROCHE GUATEMALA SOCIEDAD ANONIMA                     33220952.00
79839665  DISGUA DISTRIBUIDORA GUATEMALTECA, SOCIEDAD ANONIMA            22437500.00
84449497  BIOLOGICOS Y FARMACEUTICOS, SOCIEDAD ANONIMA                   21423233.88
12511862  OXI-RENT SOCIEDAD ANONIMA                                      20618674.31
22345388  J.I. COHEN, SOCIEDAD ANONIMA                                   19519747.35
31360831  VICLASA, SOCIEDAD ANONIMA                                      13062357.00
4556984   COMPAÑIA DE EQUIPO MEDICO-HOSPITALARIO, SOCIEDAD ANONIMA       12347737.53
82882363  SEIJIRO YAZAWA IWAI GUATEMALA SOCIEDAD ANONIMA                 10386925.00
53891236  QUALITY GROUP, SOCIEDAD ANONIMA                                 7702586.31
39254224  CETO,CETO,,MATEA,ENGRACIA                                       756440

## Top 20 concursos con mayores montos

In [129]:
nog_monto = result_red.groupby(["NOG CONCURSO", "DESCRIPCIÓN"])
print(nog_monto.MONTO.sum().sort_values(ascending=False).head(20))



NOG CONCURSO  DESCRIPCIÓN                                                                                                                                                                                                                                                                                                     
12330671      ADQUISICIÓN DE EQUIPO DE VENTILACION MECANICA Y ANESTESIA REQUERIDO POR EL INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL (ESTADO DE EXCEPCIÓN 5-2020 COVID-19) LISTADO No. 18/2020\n\n                                                                                                                                 29130000.00
12275344      ADQUISICIÓN DE VARIOS MEDICAMENTOS REQUERIDOS POR DISTINTAS UNIDADES MÉDICAS DEL INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL (ESTADO DE EXCEPCIÓN 5-2020 COVID-19) LISTADO No. 03/2020                                                                                                                               25466989.72
12289418     